<a href="https://colab.research.google.com/github/donghuna/AI-Expert/blob/main/%EC%9D%B4%EC%9E%AC%EC%9C%A4/%5B240705%5D_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


강의자료링크: https://docs.google.com/presentation/d/1n_IBA9Vp_aQK8e5Wu152UW5ZUPdJIlJc/edit?usp=sharing&ouid=112299180301409771667&rtpof=true&sd=true

- 위 자료에 이어 DPO 알고리즘을 직접 실행해보는 코드입니다.


Python에는 머신러닝 모델 학습을 간편하게 진행할 수 있도록 하는 많은 라이브러리들이 존재합니다. 우리는 huggingface와 파이토치를 이용하여 간편하게 DPO를 학습합니다.

### 1. 라이브러리 설치 및 모듈 불러오기

In [ ]:
# 필요한 라이브러리들을 먼저 설치 합니다.
# It takes about 4 minutes

# It might cause an error. But it's okay!
!pip install -q -U bitsandbytes
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl==0.8.6
!pip install deepspeed==0.13.1

In [ ]:
# 필요한 모듈을 로드 합니다.
import torch
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from trl import DPOTrainer

- 아래 코드는 이미 누군가가 학습해놓은 모델과 토크나이저를 불러오는 코드입니다. 우리는 GPT2 모델을 사용합니다

In [ ]:
# It takes about 1 ~ 2 minutes

base_model = "openai-community/gpt2"

model = AutoModelForCausalLM.from_pretrained(base_model, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(base_model, add_special_tokens=True)

In [ ]:
print("Special Tokens:", tokenizer.special_tokens_map)

Special Tokens: {'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}


#### 2. DPO 학습

In [ ]:
# colab에서 학습을 위해 huggingface에서 간단한 DPO 데이터셋을 하나 불러오겠습니다.

dataset = load_dataset("jondurbin/truthy-dpo-v0.1")


In [ ]:
# 데이터셋 구조 보기

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
        num_rows: 1016
    })
})

In [ ]:
# 실제 데이터셋 관찰

dataset['train'][0]

{'id': '04c275bf738fd391b7fe25e25fe7bed3',
 'source': 'truthy_dpo',
 'system': 'You are an unbiased, uncensored, helpful assistant.',
 'prompt': "What's the nearest national park to you?",
 'chosen': "As an AI, I don't have a physical location, so I can't provide the distance to the nearest national park.",
 'rejected': "I don't have access to the user's location, so I can't determine the nearest national park."}

In [ ]:
# 학습용 프롬프트를 만들어 보겠습니다.

def generate_prompt(example):
    prompt = example['prompt']
    rejected = example['rejected']
    chosen = example['chosen']

    example['prompt'] = f"{prompt}<|endoftext|>\n"
    example['rejected'] = f"{rejected}<|endoftext|>"
    example['chosen'] = f"{chosen}<|endoftext|>"

    return example

In [ ]:
transformed_dataset = dataset.map(generate_prompt)

In [ ]:
# 이제 학습을 위해 train과 test 데이터셋으로 나눠줍니다.
dataset = transformed_dataset['train'].train_test_split(test_size=0.1)

In [ ]:
# 아까와는 다르게, train / validation split 되어있음을 확인할 수 있습니다.
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
        num_rows: 914
    })
    test: Dataset({
        features: ['id', 'source', 'system', 'prompt', 'chosen', 'rejected'],
        num_rows: 102
    })
})

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
# Training Arguments를 설정합니다.

training_args = TrainingArguments(
        output_dir="./outputs",
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_32bit",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        per_device_eval_batch_size=4,
        logging_steps=50,
        learning_rate=5e-7,
        eval_steps=50,
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        save_strategy="no"

)

# Huggingface trainer 라이브러리에서 DPO 알고리즘을 적용한 DPO trainer 를 별도로 제공합니다.

trainer = DPOTrainer(
    model,
    ref_model=None,
    args=training_args,
    beta=0.1,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    max_prompt_length=512,
    max_length=768,
)

# 여기서 "beta"는 DPO Loss의 온도이며 일반적으로 0.1 ~ 0.5 범위입니다.
# 이것은 beta가 작을수록 레퍼런스 모델을 무시한다는 의미로 레퍼런스 모델에 얼마나 많은 관심을 기울이는지를 나타냅니다.

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/914 [00:00<?, ? examples/s]

Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [ ]:
# 학습 시작

trainer.train()

# DPO 학습의 메트릭이 의미 하는 바는 다음과 같습니다.
# Rewards/chosen : 베타로 스케일링된 선택된 응답의 정책 모델과 레퍼런스 모델의 로그 확률 간의 평균 차이
# Rewards/rejected : 정책 모델과 거부된 응답에 대한 레퍼런스 모델의 로그 확률 간의 평균 차이(베타 스케일)
# Rewards/accuracies : 선택한 보상이 해당 거부된 보상보다 높은 빈도의 평균
# Rewards/margins : 선택한 보상과 해당 거부된 보상 간의 평균 차이

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen
50,0.665800,0.632830,0.003187,-0.127538,0.942308,0.130725,-232.497803,-167.729843,-105.271919,-105.167885
100,0.596400,0.577580,0.000829,-0.270499,0.951923,0.271329,-233.927414,-167.753418,-105.216461,-105.083138
150,0.535800,0.550431,-0.001330,-0.352153,0.951923,0.350822,-234.743927,-167.775009,-105.182892,-105.026505
200,0.514400,0.541014,-0.001734,-0.381820,0.951923,0.380086,-235.040619,-167.779068,-105.171791,-105.005730


TrainOutput(global_step=229, training_loss=0.5690554531380599, metrics={'train_runtime': 201.9427, 'train_samples_per_second': 4.526, 'train_steps_per_second': 1.134, 'total_flos': 0.0, 'train_loss': 0.5690554531380599, 'epoch': 1.0})



### Reference

#### [SFT code - Medium](https://medium.com/@csakash03/fine-tuning-llama-2-llm-on-google-colab-a-step-by-step-guide-cf7bb367e790)
#### [RLHF, DPO explanation & code - Devocean](https://devocean.sk.com/blog/techBoardDetail.do?ID=165903&boardType=techBlog)
#### [RLHF explanation - IBM](https://www.ibm.com/kr-ko/topics/rlhf)
#### [RLHF explanation - huggingface](https://huggingface.co/blog/rlhf)

#### [DPO paper](https://arxiv.org/abs/2305.18290)
#### [DPO explanation & code - huggingface](https://huggingface.co/blog/pref-tuning)